In [1]:
import os 

In [2]:
%pwd

'd:\\mlops\\end-to-end-mlops-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\mlops\\end-to-end-mlops-project'

In [5]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/Adi123XD/end-to-end-mlops-project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='Adi123XD'
os.environ['MLFLOW_TRACKING_PASSWORD']='30d8e924fb35865a4187f675e6bd4a5fb5d87221'

In [6]:
print(os.getenv("MLFLOW_TRACKING_USERNAME"))

Adi123XD


In [33]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig :
  root_dir : Path
  test_data_path : Path
  model_path : Path
  metrics_file_name : Path
  all_params : dict 
  mlflow_uri : str
  target_column : str

In [34]:
print(os.getenv('MLFLOW_TRACKING_URI'))

https://dagshub.com/Adi123XD/end-to-end-mlops-project.mlflow


In [35]:
from mlProject import logger
from mlProject.utils.common import read_yaml,create_directories

In [36]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath =Path('D:\mlops\end-to-end-mlops-project\config\config.yaml'),
        params_filepath = Path("D:\mlops\end-to-end-mlops-project\params.yaml"),
        schema_filepath = Path("D:\mlops\end-to-end-mlops-project\schema.yaml")):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self )-> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])
        model_trainer_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path= config.test_data_path , 
            model_path=config.model_path,
            metrics_file_name=config.metrics_file_name,
            all_params=params,
            target_column=schema.name,
            mlflow_uri='https://dagshub.com/Adi123XD/end-to-end-mlops-project.mlflow'

        )
        return model_trainer_config

In [37]:
import pandas as pd
import joblib 
import mlflow
from urllib.parse import urlparse
import numpy as np
from sklearn.metrics import mean_absolute_error , mean_squared_error, r2_score
import mlflow.sklearn
from mlProject.utils.common import save_json

In [38]:
class ModelEvaluation :
    def __init__(self, config : ModelEvaluationConfig):
        self.config  = config

    def eval_metrics (self , actual , pred):
        rmse = np.sqrt(mean_squared_error(actual , pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual , pred)
        return rmse , mae , r2

    def log_into_mlflow ( self ):
        test = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        X_test = test.drop([self.config.target_column],axis =1)
        y_test  = test[[self.config.target_column]]
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_quantities = model.predict (X_test)
            (rmse , mae , r2) = self.eval_metrics(y_test , predicted_quantities)

            #Saving the metircs score 
            score = {'rmse':rmse , "mae" : mae , 'r2': r2}
            save_json(path = Path (self.config.metrics_file_name), data=score)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse" , rmse)
            mlflow.log_metric("mae" , mae)
            mlflow.log_metric("r2" , r2)
            if (tracking_url_type_store != "file"):
                mlflow.sklearn.log_model(model , "model",registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model , "model")


In [40]:
try: 
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e :
    logger.exception(e)
    raise e

[2024-03-29 16:52:46,498: INFO : common : yaml file: D:\mlops\end-to-end-mlops-project\config\config.yaml loaded successfully]
[2024-03-29 16:52:46,501: INFO : common : yaml file: D:\mlops\end-to-end-mlops-project\params.yaml loaded successfully]
[2024-03-29 16:52:46,508: INFO : common : yaml file: D:\mlops\end-to-end-mlops-project\schema.yaml loaded successfully]
[2024-03-29 16:52:46,512: INFO : common : created directory at: artifacts]
[2024-03-29 16:52:46,515: INFO : common : created directory at: artifacts/model_evaluation]


[2024-03-29 16:52:47,528: INFO : common : json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/03/29 16:53:04 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 4
Created version '4' of model 'ElasticnetModel'.
